## Web scraping assignment 3(using Selenium)

In [1]:
#Let's first install the Selenium library
!pip install selenium

In [3]:
#Now let's import the required libraries
#Importing Selenium
import selenium

#Importing pandas
import pandas as pd

#Importing Selenium webdriver 
from selenium import webdriver

#Importing time for sleep()
import time

#Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# importing regular expression
import re

In [4]:
#Creating driver variable and connecting to the web driver
driver = webdriver.Chrome("chromedriver.exe")

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [5]:
#Getting amazon page
driver.get('https://www.amazon.in/')

#Asking input from user
search_item= input('Enter the product that has to be searched : ')

Enter the product that has to be searched : guitar


In [6]:
#Scraping search bar 
search_bar = driver.find_element_by_id('twotabsearchtextbox')   
search_bar.clear()

#sending keys which user has input
search_bar.send_keys(search_item)

#clicking on search icon
search_button = driver.find_element_by_id('nav-search-submit-button')
search_button.click()
time.sleep(2)

### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [12]:
#collecting all the Product URLS
urls = []   #empty list
for i in range(0,3):  #only first 3 pages
    Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the "user input item"
    for i in Page_urls:   
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element_by_xpath("//li[@class='a-last']//a") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [13]:
#checking length of all the items URL. There are 60 items per page.
len(urls)

180

In [16]:
#Making empty lists and scraping the required data
Product_name = []
Brand_name= []
Ratings = []
No_Ratings = []
Price = []
Return = []
Expected_Delivery = []
Availability = []
Other_Details = []

#Looping through URLs
for i in urls:
    driver.get(i)
    time.sleep(1)
    
    
    #Scraping data for product name
    try:
        prod=driver.find_element_by_xpath("//span[@id='productTitle']")
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping data for brand name
    try:
        brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
     #Scraping data for Ratings
    try:
        rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
        Ratings.append(rat.get_attribute("title"))   
    except NoSuchElementException as e:
        Ratings.append("-")
        
        
    #Scraping data for No of Ratings
    try:
        no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']/span")
        No_Ratings.append(no_rat.text)
    except NoSuchElementException as e:
        No_Ratings.append("-")
        
        
    #Scraping data for Price
    try:
        pri=driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping data for Return/Exchange
    try:
        ret=driver.find_element_by_xpath("//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping data for Expected_Delivary
    try:
        delivary=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping data for Availability
    try:
        avai=driver.find_element_by_xpath("//div[@id='availability']/span")
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
        
        
    #Scraping data for Other_Details
    try:
        details=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        Other_Details.append(details.text)
    except NoSuchElementException as e:
        Other_Details.append("-")


In [17]:
#Creating the dataframe from the scraped data
Laptop=pd.DataFrame({})
Laptop['Name'] = Product_name
Laptop['Brand'] = Brand_name
Laptop['Rating'] = Ratings
Laptop['No of Ratings'] = No_Ratings
Laptop['Price'] = Price
Laptop['Return'] = Return
Laptop['Expected_Delivery'] = Expected_Delivery
Laptop['Availability'] = Availability
Laptop['Other_Details'] = Other_Details
Laptop['Urls'] = urls
Laptop

,Name,Brand,Rating,No of Ratings,Price,Return,Expected_Delivery,Availability,Other_Details,Urls
0,Medellin MED-BLU-C Linden Wood Acoustic Guitar,Brand: medellin,3.9 out of 5 stars,523 ratings,"₹2,399.00",7 Days Replacement,"Friday, Aug 27",In stock.,Material: Wood\nColour: Blue\nAcoustic Guitar,https://www.amazon.in/gp/slredirect/picassoRed...
1,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,3.9 out of 5 stars,"12,218 ratings","₹2,649.00",7 Days Replacement,"Thursday, Aug 26",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/gp/slredirect/picassoRed...
2,Intern INT-38C Acoustic Guitar Kit (Sunburst) ...,Visit the Intern Store,3.9 out of 5 stars,"6,625 ratings","₹2,390.00",7 Days Replacement,"Thursday, Aug 26",In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,Visit the Kadence Store,3.9 out of 5 stars,984 ratings,"₹6,999.00",7 Days Replacement,"Friday, Aug 27",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,3.9 out of 5 stars,"12,218 ratings","₹2,649.00",7 Days Replacement,"Thursday, Aug 26",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
175,"JUAREZ Arpeggio 41 Inch Acoustic Guitar Kit, S...",Brand: JUAREZ,4.1 out of 5 stars,27 ratings,"₹5,265.00",7 Days Replacement,"Thursday, Aug 26",In stock.,"Juarez Arpeggio 41 Inch Acoustic Guitar Kit, S...",https://www.amazon.in/JUAREZ-Arpeggio-Acoustic...
176,"Blueberry B-40, 40"" Acoustic Guitar Kit with B...",Visit the blueberry Store,1.0 out of 5 stars,1 rating,"₹3,595.00",7 Days Replacement,"Sunday, Aug 29",In stock.,"Material: Top: Spruce wood, Side & Back: - lin...",https://www.amazon.in/gp/slredirect/picassoRed...
177,"Guitar Straight Edge, Notched Straight Edge Lu...",Brand: menfad,-,-,"₹2,939.00",10 Days Returnable,Sep 8 - 11,In stock.,"Made of premium copper-zinc alloy material, st...",https://www.amazon.in/gp/slredirect/picassoRed...
178,"JUAREZ Arpeggio 41 Inch Acoustic Guitar Kit, S...",Brand: JUAREZ,4.0 out of 5 stars,7 ratings,"₹5,210.00",7 Days Replacement,"Thursday, Aug 26",In stock.,"Juarez Arpeggio 41 Inch Acoustic Guitar Kit, S...",https://www.amazon.in/gp/slredirect/picassoRed...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [4]:
#firstly, get the webpage https://images.google.com/
driver.get('https://images.google.com/')

In [7]:
#Get the search bar
search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_bar.clear()

# A. FRUITS
#Sending keys to search bar
search_bar.send_keys('fruits')

#Clicking search button
search_button = driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_button.click()
time.sleep(3)

In [47]:
#Get all the webelements for fruits images
fruits_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

#Scraping fruits images
fruits_list = [] #empty list to scape data
for i in fruits_tags:       
    image = i.get_attribute("src")    #get image source from webelements
    if image != None:                 #if there is no none value
        if image[:5]=='https':        #starts with https 
            fruits_list.append(image) #append the result in list
fruits_list

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyVDjHPnxb-fUSfdPQmLokeax27bUXHHkT4g&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRd_GNOtXoobR03rU2buQyAcerEWCMA5QxIoA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREf50eNWUXvnoP0sP9x8SZUILcwQHHImfm1A&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaeGJ_PSJlNLxm6sUANm3yiU0_G-x4-FAsmA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTsRLK-La6p9LPjx5MkCD7KjZ2PFQY8XNX0YA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6SjYxZXy_JKZ6NhUw1ZlY2uLry91NcHxN6A&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsZyKU4pEUcVP2x_GtXrgZ8kRbLhim4vwD5Q&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSU8908ZiQ-8SkCe--IgWNFs9AnpUIRovgcfQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQle5gAzia23iAHTi1cM7lyxt-VpNMzHbAV9A&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT68b0O5w

In [52]:
# B. CARS

#Get the new search bar as the page is changed 
search_bar = driver.find_element_by_id("REsRA")
search_bar.clear()

#Sending keys to search bar
search_bar.send_keys('cars')

#Clicking search button
search_button = driver.find_element_by_id("BIqFsb")
search_button.click()
time.sleep(3)

In [55]:
#Get all the webelements for cars images
cars_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

#Scraping cars images
cars_list = [] #empty list to scape data
for i in cars_tags:       
    image = i.get_attribute("src")    #get image source from webelements
    if image != None:                 #if there is no none value
        if image[:5]=='https':        #starts with https 
            cars_list.append(image) #append the result in list
cars_list

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxbjXwkwVr-14A0KU5__5zZb-b5nUEyJv2Rg&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQFQD8FHJ2cf_3_trxiRLElMRBl6yHz--OmJA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSyMurcEluQ5blOUZgaUN_RAAsFJYgp37mLXQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQx6Y26YOjgzDm8RNQL3V0oa3ZqrYhfHmnLyA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPTXYOJCG-NmDxyu_Rzo6oO5OMgCdRWgTkkA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWGvBIMKjRqiKRVEeGP4SU5-ODjSYCNNzXBw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwJ-3rQj6ssAkw_CLSFd0slbAK5oy2whAakw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXkhUxznR2Y_ygHH1Hjzlknq62zFof-H5PoQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQjFcOsHo5tCzy367Ot3CZDD_FUtiJHtWYBQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRh9s1Eyu

In [58]:
# C. MACHINE LEARNING

#Get the new search bar as the page is changed 
search_bar = driver.find_element_by_id("REsRA")
search_bar.clear()

#Sending keys to search bar
search_bar.send_keys('machine learning')

#Clicking search button
search_button = driver.find_element_by_id("BIqFsb")
search_button.click()
time.sleep(3)

In [60]:
#Get all the webelements for cars images
ml_tags = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")

#Scraping ML images
ml_list = [] #empty list to scape data
for i in ml_tags:       
    image = i.get_attribute("src")    #get image source from webelements
    if image != None:                 #if there is no none value
        if image[:5]=='https':        #starts with https 
            ml_list.append(image) #append the result in list
ml_list

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSav49NDq1Pa_S4JFC0wuCjA8AsjVgSuSm5UA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5LFpAD-FPmniqb8OCMo64l4q87A37IN6UMQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRnnXZkPIuKunti63oDZ3qPDF-9YlYYuJaTmQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQh4HzGIRo74WpbQfcr6SFZFkcsiykKzX5A-w&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfN12oiCbkM3yQNEyPU1BU2oWCHh3gEGKNcA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ-qD2rRXFGMce45la7pokLMZpwRAad6sZKAw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSp2BlkHDrMpBa5pOZAgpzkL8RMmvgEU5I_Aw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRyUsvp8lFRGIPJc6R5KIfzr2Y-3uklW1hZLg&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRkU62g7Rhz3aNupBoOc6pizY-19hH9-MJKA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqX-g9A6

In [65]:
#Creating the dataframe from the scraped data and taking only first 100 entries
df = pd.DataFrame({'Fruits images':fruits_list[:100],
                  'Cars images':cars_list[:100],
                  'Machine Learning images':ml_list[:100]})
df

,Fruits images,Cars images,Machine Learning images
0,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [84]:
# Activating the chrome browser
item = input(" Enter the name of Smartphone that has to be searched : ")
driver = webdriver.Chrome("chromedriver.exe") 

#get the web page with given url
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

 Enter the name of Smartphone that has to be searched : samsung


In [85]:
#Clicking on login close icon
login_X_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()

In [86]:
#giving input key word to search bar
serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

In [87]:
# Fetching urls of phones coming on 1st page
page1_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [88]:
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Display Resolution"] = []
Smartphones["Processor"] = []
Smartphones["Processor Cores"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [89]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
    
    #Scraping display resolution data
    try:
        dires_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if dires_tags.text != "Display Features" : raise NoSuchElementException
        disp_reso = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')    
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
    
    
    
    #Scraping Processor data
    try:
        pro_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_tags.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
        
    
    
    #Scraping Processor core data    
    try:                                                                                     
        core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_tags.text != "Processor Core" :
            core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_tags.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        Smartphones['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
    
    
    
    #Scraping battery capacity data
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

In [90]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,SAMSUNG,M31,Ocean Blue,6 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹16,389",https://www.flipkart.com/samsung-m31-ocean-blu...
1,SAMSUNG,Guru 1200,White,1000,NA,No,-,3.81 cm (1.5 inch),128 x 128 Pixels,-,-,800 mAh,"₹1,395",https://www.flipkart.com/samsung-guru-1200/p/i...
2,SAMSUNG,Guru Music 2,Blue,Yes,NA,-,-,5.08 cm (2 inch),128 x 160 Pixels,-,-,800 mAh,"₹1,625",https://www.flipkart.com/samsung-guru-music-2/...
3,SAMSUNG,Metro 313 Dual Sim,Grey,16 MB,2 MB,0.3MP Rear Camera,-,5.08 cm (2 inch),128 x 160 Pixels,-,-,1000 mAh,"₹2,330",https://www.flipkart.com/samsung-metro-313-dua...
4,SAMSUNG,Guru Music 2,White,16 GB,NA,NA,0MP Front Camera,5.08 cm (2 inch),128 x 160 Pixels,NA 0 Single Core 208MHz,Single Core,800 mAh,"₹1,935",https://www.flipkart.com/samsung-guru-music-2/...
5,SAMSUNG,Guru FM Plus SM-B110E/D,Dark Blue,Yes,NA,-,-,3.81 cm (1.5 inch),128 x 128 Pixels,-,-,800 mAh,"₹1,549",https://www.flipkart.com/samsung-guru-fm-plus-...
6,SAMSUNG,Guru Music 2,Black,Yes,NA,-,-,5.08 cm (2 inch),128 x 160 Pixels,-,-,800 mAh,"₹1,945",https://www.flipkart.com/samsung-guru-music-2/...
7,SAMSUNG,Guru FM Plus,Gold,Yes,NA,-,-,3.81 cm (1.5 inch),128 x 128 Pixels,-,-,800 mAh,"₹1,645",https://www.flipkart.com/samsung-guru-fm-plus/...
8,SAMSUNG,Guru FM Plus SM-B110E/D,White,1000,NA,No,-,3.81 cm (1.5 inch),128 x 128 Pixels,-,-,800 mAh,"₹1,549",https://www.flipkart.com/samsung-guru-fm-plus-...
9,SAMSUNG,Guru FM Plus,Black,Yes,128 MB,-,-,3.81 cm (1.5 inch),128 x 128,-,Single Core,800 mAh,"₹1,642",https://www.flipkart.com/samsung-guru-fm-plus/...


In [91]:
#saving data to csv
df.to_csv('Flipkart_{}.csv'.format(item))

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [104]:
#firstly, get the webpage https://www.google.co.in/maps
driver.get('https://www.google.co.in/maps')
time.sleep(1)

In [105]:
#Asking user to enter a city name
city = input('Enter a city name: ')

#Get the search bar
search_bar = driver.find_element_by_id("searchboxinput") 
search_bar.clear()

#Sending keys to search bar
search_bar.send_keys(city)

#Clicking search button
search_button = driver.find_element_by_id("searchbox-searchbutton")
search_button.click()
time.sleep(3)

Enter a city name: nagpur


In [117]:
#Scraping latitude & longitude values from current URL
try:
    url_string = driver.current_url           #get current url               
    print("City URL: ", url_string)           
    latitude_longitude = re.findall(r'@(.*)data',url_string)   #regex to get data between @ and data
    lat_lng_list = latitude_longitude[0].split(",")     #splitting using ,
    if len(lat_lng_list)>=2:                            
        latitude = lat_lng_list[0]                      
        longitude = lat_lng_list[1]
    print("Latitude: {} \nLongitude: {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", e)

City URL:  https://www.google.co.in/maps/place/Nagpur,+Maharashtra/@21.1613481,78.9320776,11z/data=!3m1!4b1!4m5!3m4!1s0x3bd4c0a5a31faf13:0x19b37d06d0bb3e2b!8m2!3d21.1458004!4d79.0881546
Latitude: 21.1613481 
Longitude: 78.9320776


### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [120]:
#firstly, get the webpage https://www.trak.in
driver.get('https://www.trak.in')
time.sleep(1)

In [121]:
#Getting xpath for funding deals
fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [122]:
#Creating empty lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry OR Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []

In [123]:
#Clicking on tablepress
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Scraping data of Date
    date_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for date in date_tags:
        fund_deals['Date'].append(date.text)

    # Scraping data of Startup Name
    name_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in name_tags:
        fund_deals['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    ind_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind_tags:
        fund_deals['Industry OR Vertical'].append(n.text)
    
    # Scraping data of Sub-Vertical
    sv_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sv_tags:
        fund_deals['Sub-Vertical'].append(sv.text)

    # Scraping data of Location
    loc_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        fund_deals['Location'].append(loc.text)
    
    # Scraping data of Investor
    inv_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        fund_deals['Investor'].append(inv.text)
        
    # Scraping data of Investment Type
    invt_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        fund_deals['Investment Type'].append(invt.text)
    
    # Scraping data of Amount
    amt_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        fund_deals['Amount(in USD)'].append(amt.text)

In [124]:
#Creating the dataframe from the scraped data
fund_df = pd.DataFrame(fund_deals)
fund_df

,Date,Startup Name,Industry OR Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [126]:
#Saving data to csv
fund_df.to_csv("Trak_in.csv")

### 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [125]:
#firstly, get the webpage https://www.digit.in
driver.get('https://www.digit.in')
time.sleep(1)

In [128]:
#searching for best laptop
best_gam_lap = driver.find_element_by_xpath("//div[@class='Listbrand']//ul//li[10]//a").click()
time.sleep(3)

In [129]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []


# Scraping the data of laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
#Scraping the data of operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


#Scraping data of memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


#Scraping data of weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [130]:
#Creating the dataframe from the scraped data
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [131]:
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A
1,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,Rs 341990
2,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A
3,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,Rs 164990
4,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,Rs 76988
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,Rs 185000
6,Acer Aspire 7 gaming,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,Rs 64370


In [132]:
#Saving dataset to csv
Gaming_Laptop.to_csv("Gaming_laptops.csv")

### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [31]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(1)

#get the specified url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(1)

#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billionaire  
bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
bill.click()
time.sleep(1)

#select world billionaire  
world_bill= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)

In [33]:
#Scraping required data from the web page
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #Scraping the data of rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    
    
    #Scraping the data of names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    
    
    #Scraping data of age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    
    
    #Scraping data of citizenship
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    
    
    #Scraping data of source of income
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    
    
    #Scraping data of Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    
    break
    

#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [34]:
#Creating the dataframe from the scraped data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


In [35]:
#Saving dataset to csv
Billionaires.to_csv("Billionaries.csv")

### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [136]:
#firstly, get the webpage https://www.youtube.com
driver.get('https://www.youtube.com')
time.sleep(1)

In [137]:
#finding element for search bar
search_bar = driver.find_element_by_id('search')
search_bar.send_keys("GOT")  #entering Video name
time.sleep(1)

#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [138]:
#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()
time.sleep(1)

In [140]:
# 1000 time we scroll down to generate more Comments
for _ in range(100):
    driver.execute_script("window.scrollBy(0,1000)")

In [141]:
#make empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

#scraping the data of comments
cm_tags = driver.find_elements_by_id("content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)

# scraping the data of time when comment was posted
tm_tags = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for tm in tm_tags:
    Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)

# scraping the data of comment likes
like_tags = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])

In [142]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments
Youtube['Comment_time'] = comment_time
Youtube['Comment upvotes'] = No_of_Likes
Youtube

,Comments,Comment_time,Comment upvotes
0,she was so amazing. how could they do this to ...,8 months ago,11K
1,I swear this was probably one of the most BOSS...,4 months ago,3.1K
2,the way she just turns around and says “dracar...,5 months ago,2.8K
3,“But harm no child” season 8 forgets only this...,3 months ago,1.5K
4,Anyone else think that the first unsullied to ...,7 months ago,1.8K
...,...,...,...
655,2:56 ... Missandei's face is so amazing. She h...,2 years ago,63
656,MY FAVORİTE SCENE ! DRACARYS 🔥,7 months ago,1
657,How a show that should belong to the Hall of L...,4 months ago,
658,Watching after last season director unnecessar...,6 months ago,


In [143]:
#Saving dataset to csv
Youtube.to_csv("Youtube_data.csv")

### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [18]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

#get the web page with given url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(5)

In [20]:
#locating the location search bar
search_loc = driver.find_element_by_id('search-input-field')

#write London in search bar
search_loc.send_keys("London")
time.sleep(5)

#select london
london = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(5)


# do click on search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()
time.sleep(3)

In [21]:
#lets find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []


#Scraping the requered informations
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(3)
    #fetching hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #fetching distance from city centre
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #fetch privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #fetching dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #fetching facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #lets fetch url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #lets click on show more button for description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #fetching ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    #fetching total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #fetch overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #fetch property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [22]:
#Creating dataframe
dff = pd.DataFrame({})
dff['Hostel_Name'] = hostel_name
dff['Distance fron city centre'] = distance
dff['Ratings'] = rating
dff['Total_reviews'] = reviews
dff['Overall Reviews'] = over_all
dff['Privates from price'] = pvt_prices
dff['Dorms from price'] = dorms_price
dff['Facilities'] = facilities[:83]
dff['Description'] = description
dff

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
0,London Waterloo Hostel,0.7km from city centre,7.5,2409,Very Good,Rs5586,Rs1615,"Free WiFi, Follows Covid-19 sanitation guidance","73 Lambeth Walk, London, London, England"
1,Barmy Badger Backpackers,5.5km from city centre,10,1653,Superb,Rs5586,Rs1615,"Free WiFi, Follows Covid-19 sanitation guidance","17 Longridge Road, Earls Court, London, England"
2,Wombat's The City Hostel London,3.6km from city centre,9.2,13143,Superb,Rs5586,Rs1615,"Free WiFi, Follows Covid-19 sanitation guidance","7 Dock Street, London, England"
3,Prime Backpackers Angel,3.6km from city centre,10,513,Superb,Rs5586,Rs1615,"Free WiFi, Follows Covid-19 sanitation guidance","333 City Road, 333 City Road, London, England"
4,St Christopher's Village,1.8km from city centre,8.9,10830,Fabulous,Rs5586,Rs1615,"Free WiFi, Follows Covid-19 sanitation guidance","165 Borough High Street, London, England"
...,...,...,...,...,...,...,...,...,...
78,The Dover,Hotel - 1.9km from city centre,-,0,No Rating,Rs6500,-,"Free WiFi, Follows Covid-19 sanitation guidance","44 Belgrave Road, London, England"
79,Park Hotel Essex,Hotel - 24.1km from city centre,-,0,No Rating,Rs6500,-,"Free WiFi, Follows Covid-19 sanitation guidance","327 Cranbrook Road, Ilford, London, England"
80,Cranbrook Hotel,Hotel - 14.8km from city centre,-,0,No Rating,Rs6500,-,"Free WiFi, Follows Covid-19 sanitation guidance","22/24 Coventry Road, Ilford, London, England"
81,St. Athans,Bed and Breakfast - 2.9km from city centre,-,0,No Rating,Rs6500,-,"Free WiFi, Follows Covid-19 sanitation guidance","20 Tavistock Place, Russell Square, London, En..."


In [ ]:
#Saving dataset to csv
dff.to_csv("London_Hostels.csv")